In [1]:
# Importing Python packages

import pandas as pd
import numpy as np
import plotnine as p9
import os
import geopandas as gpd
from pathlib import Path


# Data ingest

In [8]:
# Reading in masterplan data

data_masterplan_input = pd.read_csv('mp_data.csv')
data_masterplan_input.head(1)

,Municipality,ProjectID,Include in Summary,Project Name,DistrictID,District,Regional_Subregional Centre_District,District Centre,Centre or Non Centre,Approved,...,2031,2032,2033,2034,2035,2036,2037,2038,2039,2040
0,Abu Dhabi,23037.4,No,Bloom District Phase 2,400,ZAYED CITY,ZAYED CITY,0,NonCentre,Yes,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
# Reading in TMP 2040 data

tmp_2040_input = pd.read_csv('20240605_TAZ_TMP_2040.csv')
tmp_2040_input.head(1)

,Z,Unnamed: 1,Master plan,PRODSECTOR,AREATYPE,AREATYPE2,UPC_PRECIN,PRECINCT_,HH,POP_NONLAB,...,SECT_INT,REG,TOD_A,TOD_P,HGV_PERMIT,ADSICDIST,DISTNAME,TMP,UDM_DIST,MP_ProjectID
0,1.0,ADM,MP,2.0,9.0,11.0,28.0,Saadiyat Island,283.0,2150.0,...,0.0,1.0,0.0,0.0,1.0,67.0,SADIYAT ISLAND,Saadiyat Lagoons TMP - Saadiyat,AL SAADIYAT ISLAND,1007.42.4


In [15]:
# Reading in land use crosswalk

land_use_crosswalk = pd.read_csv('taxonomy_crosswalks.csv')
land_use_crosswalk = land_use_crosswalk[land_use_crosswalk['CROSSWALK'] == 'MP to TMP 2040 land use']
land_use_crosswalk.head(1)

,CROSSWALK,VARIABLE_1,VARIABLE_2,NOTES
17,MP to TMP 2040 land use,Residential (GFA),RES_GFA,NaN


# Something goes here

In [16]:
# Creating list for selecting development quantity columns from the masterplan datasheet

tmp_land_uses = land_use_crosswalk['VARIABLE_2'].drop_duplicates().tolist()
tmp_land_uses

['RES_GFA',
 'OFFICE_GFA',
 'RETAIL_GFA',
 'HOTELROOMS',
 'IND_GFA',
 'SCHOOL_GFA',
 'MED_GFA',
 'OTHER_GFA']

In [17]:
data_masterplan_input

,Municipality,ProjectID,Include in Summary,Project Name,DistrictID,District,Regional_Subregional Centre_District,District Centre,Centre or Non Centre,Approved,...,2031,2032,2033,2034,2035,2036,2037,2038,2039,2040
0,Abu Dhabi,23037.4,No,Bloom District Phase 2,400,ZAYED CITY,ZAYED CITY,0,NonCentre,Yes,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Abu Dhabi,1238.0,Yes,Marasy,1300,AL BATEEN,AL BATEEN,0,NonCentre,Yes,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Abu Dhabi,1113.2.1,Yes,IKEA,1330,YAS ISLAND,YAS ISLAND,0,NonCentre,Yes,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Abu Dhabi,ABU MREIKHAH_2,Yes,ABU MREIKHAH_2,1311,ABU MREIKHAH,ABU MREIKHAH,0,NonCentre,No,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Abu Dhabi,Rabdan,Yes,Rabdan,NaN,NaN,NaN,0,NonCentre,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
360,Abu Dhabi,Labor Accommodation 88,NaN,JARN YAFOUR,NaN,JARN YAFOUR Labor Accommodation 88,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
361,Abu Dhabi,Labor Accommodation 89,NaN,JARN YAFOUR,NaN,JARN YAFOUR Labor Accommodation 89,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
362,Abu Dhabi,Labor Accommodation 90,NaN,AL BAHYAH,NaN,AL BAHYAH Labor Accommodation 90,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
363,Abu Dhabi,Labor Accommodation 91,NaN,KHALIFA INDUSTRIAL,NaN,KHALIFA INDUSTRIAL Labor Accommodation 91,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [2]:
# Reading in baseline TAZ data

sharepoint_path = f'C:/Users/{os.environ.get( "USERNAME" )}/Buro Happold'
project_path = Path(f"{sharepoint_path}/P060927 Plan Capital 2040 - 02_Documents/11_GIS/05 Urban Models")
ium_path = os.path.join(project_path,'ium_ad.gpkg')
zone_taz_2022 = gpd.read_file(ium_path, layer='zone_taz_2022')
zone_taz_2022.head(1)


,Z,SHAPE_Length,SHAPE_Area,NUM_OF_BUILDINGS,Calc_Total_Ex_GFA,Ex_GFA,RES_GFA,Residential_Labour_GFA,OFFICE_GFA,RETAIL_GFA,...,RETMAL_PRO,MEDCLI_PRO,MEDHOS_PRO,NONTER_PRO,TER_PRO,SCHPLCS_NT,SCHPLCS_TE,SCHPLCS_PR,SCHPLCS_SE,geometry
0,1044,47079.0,606869.0,679.0,297016.0,297542.0,271410.0,0.0,0.0,0.0,...,None,None,None,None,None,None,None,None,None,"MULTIPOLYGON (((270352.336 2727199.707, 269735..."


In [3]:
# Reading in taz zone delta

zone_taz_delta = gpd.read_file(ium_path, layer='zone_taz_delta')
zone_taz_delta.head(1)


,Z,ZONE,EPA,UPC_PRECIN,PRECINCT_N,ST_DIST_ID,ST_DIST_EN,UDM_DIST_I,UDM_DIST_E,REGION,...,score_component_rank_sum_rank_mp,score_component_rank_mean_rank_mp,score_component_rank_sum_rank_urban,score_component_rank_mean_rank_urban,score_development_priority,2022_taz_far,gfa_per_ha,landmodel_limit_gfa_per_ha,landmodel_limit_gfa_per_ha_delta,geometry
0,2510,2510,None,1.0,Abu Dhabi,1.0,Abu Dhabi West Coast And Islands And Mussaffah...,500.0,ABU AL ABYAD ISLAND,ADM,...,NaN,NaN,1.0,1.0,5.0,0.000003,0.026657,0.026657,-0.025877,"MULTIPOLYGON (((182767.838 2687320.951, 183001..."


In [6]:
# Reading in land-use split
land_use_crosswalk


,CROSSWALK,VARIABLE_1,VARIABLE_2,NOTES
0,TAZ to MP land use crosswalk,Residential (GFA),RES_GFA,NaN
1,TAZ to MP land use crosswalk,Residential Labour (GFA),Residential_Labour_GFA,NaN
2,TAZ to MP land use crosswalk,Office (GFA),OFFICE_GFA,NaN
3,TAZ to MP land use crosswalk,Retail (GFA),RETAIL_GFA,NaN
4,TAZ to MP land use crosswalk,Hotel (GFA),Hotel_GFA,NaN
5,TAZ to MP land use crosswalk,Retail (GFA),Commercial_Recreation_GFA,NaN
6,TAZ to MP land use crosswalk,Industrial (GFA),Industrial_GFA,NaN
7,TAZ to MP land use crosswalk,Education (GFA),SCHOOL_GFA,NaN
8,TAZ to MP land use crosswalk,Healthcare (GFA),MED_GFA,NaN
9,TAZ to MP land use crosswalk,Community Facilities (GFA),RELIGIOUS_GFA,NaN


In [5]:
tmp_2040_select_columns = ['Z','MP_ProjectID','RES_GFA','RETAIL_GFA','OFFICE_GFA','L_IND_GFA','M_IND_GFA','H_IND_GFA','SCHOOL_GFA','MED_GFA','OTHER_GFA']

In [7]:
tmp_2040 = tmp_2040_input[tmp_2040_select_columns]
tmp_2040 = tmp_2040[~tmp_2040['Z'].isna()]
tmp_2040['Z'] = tmp_2040['Z'].astype(str)
tmp_2040['TOTAL_GFA'] = tmp_2040.select_dtypes(include=[float, int]).sum(axis=1)
tmp_2040.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3476 entries, 0 to 3475
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Z             3476 non-null   object 
 1   MP_ProjectID  3476 non-null   object 
 2   RES_GFA       3476 non-null   float64
 3   RETAIL_GFA    3476 non-null   float64
 4   OFFICE_GFA    3476 non-null   float64
 5   L_IND_GFA     3476 non-null   float64
 6   M_IND_GFA     3476 non-null   float64
 7   H_IND_GFA     3476 non-null   float64
 8   SCHOOL_GFA    3476 non-null   float64
 9   MED_GFA       3476 non-null   float64
 10  OTHER_GFA     3476 non-null   float64
 11  TOTAL_GFA     3476 non-null   float64
dtypes: float64(10), object(2)
memory usage: 353.0+ KB


In [8]:
tmp_2040_per = tmp_2040.select_dtypes(include=[float, int]).div(tmp_2040['TOTAL_GFA'], axis=0)
tmp_2040_per = pd.concat([tmp_2040[['Z','MP_ProjectID']],tmp_2040_per],axis = 1)
tmp_2040_per = tmp_2040_per[tmp_2040['MP_ProjectID'] != '0'].reset_index(drop = True)
tmp_2040_per = tmp_2040_per[~tmp_2040['MP_ProjectID'].isna()].reset_index(drop = True)
tmp_2040_per

C:\Users\jtindale\AppData\Local\Temp\ipykernel_28112\1306470590.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


,Z,MP_ProjectID,RES_GFA,RETAIL_GFA,OFFICE_GFA,L_IND_GFA,M_IND_GFA,H_IND_GFA,SCHOOL_GFA,MED_GFA,OTHER_GFA,TOTAL_GFA
0,1.0,1007.42.4,0.986446,0.001520,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.012034,1.0
1,2.0,1007.42.4,0.899572,0.007761,0.000000,0.000000,0.000000,0.000000,0.075361,0.000000,0.017306,1.0
2,3.0,1007.0,0.439153,0.000000,0.560847,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0
3,4.0,1007.27,0.700105,0.022941,0.000000,0.000000,0.000000,0.000000,0.162524,0.000000,0.114430,1.0
4,5.0,1007.42.4,0.990072,0.000953,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.008975,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
1649,3472.0,1997.0,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0
1650,3473.0,1997.0,0.000000,0.873139,0.076865,0.000000,0.000000,0.000000,0.000000,0.000000,0.049995,1.0
1651,3474.0,1995.0,0.000000,0.006808,0.031058,0.207263,0.377436,0.377436,0.000000,0.000000,0.000000,1.0
1652,3475.0,1995.0,0.000000,0.002353,0.000000,0.199269,0.398537,0.398537,0.000000,0.000858,0.000445,1.0


In [11]:
tmp_2040_input.columns

Index(['Z', 'Unnamed: 1', 'Master plan', 'PRODSECTOR', 'AREATYPE', 'AREATYPE2',
       'UPC_PRECIN', 'PRECINCT_', 'HH', 'POP_NONLAB', 'WORKER', 'STUDENT',
       'LABOURER', 'POP_TOT', 'HOTELROOMS', 'RES_GFA', 'RETAIL_GFA',
       'OFFICE_GFA', 'L_IND_GFA', 'M_IND_GFA', 'H_IND_GFA', 'SCHOOL_GFA',
       'MED_GFA', 'OTHER_GFA', 'CONS_SITE', 'RETGEN_PRO', 'RETMAL_PRO',
       'MEDCLI_PRO', 'MEDHOS_PRO', 'NONTER_PRO', 'TER_PRO', 'SCHPLCS_NT',
       'SCHPLCS_TE', 'SCHPLCS_PR', 'SCHPLCS_SE', 'SECT_DST', 'SECT_FRT',
       'SECT_EXT', 'SECT_REP', 'SECT_REPMS', 'METRO_AREA', 'SHAPEAREA',
       'CENTROIDX', 'CENTROIDY', 'PARK_SEC', 'URBAN_RURA', 'LAB_CON_P',
       'LAB_OTH_P', 'LAB_IND_P', 'IND_AREA', 'PROP_INT_L', 'PROP_INT_W',
       'PROP_INT_O', 'SECT_BAL', 'SECT_INT', 'REG', 'TOD_A', 'TOD_P',
       'HGV_PERMIT', 'ADSICDIST', 'DISTNAME', 'TMP', 'UDM_DIST',
       'MP_ProjectID'],
      dtype='object')